In [ ]:
!pip install scipy==1.2 --upgrade

Requirement already up-to-date: scipy==1.2 in /usr/local/lib/python3.7/dist-packages (1.2.0)


In [ ]:
!pip uninstall statsmodels


Uninstalling statsmodels-0.10.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/statsmodels-0.10.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/statsmodels/*
Proceed (y/n)? y
  Successfully uninstalled statsmodels-0.10.2


In [ ]:
!pip install statsmodels

     |████████████████████████████████| 9.5MB 27.7MB/s 


In [ ]:
from datetime import datetime
from tqdm import tqdm
import requests
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
url = 'http://iss.moex.com/iss/engines/stock/markets/shares/boardgroups/57/'

In [ ]:
def get_number_of_candles(start_date: datetime, interval: int):
    "Получение общего количества свечей от указанной стартовой даты и интервала"
    gap = datetime.now() - start_date
    gap_in_minutes = gap.days * 24 * 60
    number_of_candles = int(gap_in_minutes / interval)
    return number_of_candles

In [ ]:
def get_candles(secid, start_date, interval):
    number_of_candles = get_number_of_candles(start_date, interval)
    candles = []
    full_url = url + 'securities/{0}/candles.json?from={1}&interval={2}'.format(secid, start_date.strftime('%Y-%m-%d'), interval)
    for number in tqdm(range(0, number_of_candles, 100)):
        r = requests.get(full_url + '&start={0}'.format(number))
        data = r.json()
        candles.extend(data['candles']['data'])
    return [v[0] for v in candles]

In [ ]:
start_date_month = datetime(2021, 4, 24)
start_date_week = datetime(2021, 5, 17)
start_date_day = datetime(2021, 5, 24)

In [ ]:
candles_month = {
    'AFLT': get_candles('AFLT', start_date_month, 60),
    'MTSS': get_candles('MTSS', start_date_month, 60),
    'OZON': get_candles('OZON', start_date_month, 60),
}

100%|██████████| 8/8 [00:02<00:00,  2.79it/s]


In [ ]:
candles_week = {
    'AFLT': get_candles('AFLT', start_date_week, 60),
    'MTSS': get_candles('MTSS', start_date_week, 60),
    'OZON': get_candles('OZON', start_date_week, 60),
}

100%|██████████| 2/2 [00:00<00:00,  2.74it/s]


In [ ]:
candles_day = {
    'AFLT': get_candles('AFLT', start_date_day, 10),
    'MTSS': get_candles('MTSS', start_date_day, 10),
    'OZON': get_candles('OZON', start_date_day, 10),
}

100%|██████████| 2/2 [00:00<00:00,  3.48it/s]


In [ ]:
def selection_of_arima_params(data):
  d = range(1, 4)
  q = range(1, 4)
  
  best_aic = float('inf')
  results = []

  for i in tqdm(d):
    for j in q:
      try:
        model = ARIMA(data, order=(i, 1, j))
        model_fit = model.fit()
      except:
        continue

      aic = model_fit.aic
      if aic < best_aic:
        best_aic = aic
        best_params = (i, 1, j)
      results.append([(i,1,j), aic])    
  return best_aic, best_params, results

In [ ]:
def get_predictions_arima(train, test, order):
    history = train
    predictions = []
    for i in tqdm(range(len(test))):
        model = ARIMA(history, order=order)
        model_fit = model.fit()
        output = model_fit.forecast()
        y_hat = output[0]
        predictions.append(y_hat)
        history.append(test[i])
    return predictions

In [ ]:
def get_predicitons_for_arima(candles, start_date, interval):
  predictions = {}
  for key in candles:
    curr_candles = candles[key]
    _, best_params, _ = selection_of_arima_params(curr_candles)
    train_length = int(len(curr_candles) * 0.7)
    train = curr_candles[:train_length]
    test = curr_candles[train_length:]
    predicts = get_predictions_arima(train, test, best_params)
    predictions[key] = [test, predicts]
  return predictions




In [ ]:
arima_preds_month = get_predicitons_for_arima(candles_month, start_date_month, 60)


 67%|██████▋   | 2/3 [00:02<00:00,  1.24it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'

100%|██████████| 3/3 [00:04<00:00,  1.36s/it]

  3%|▎         | 5/188 [00:01<00:37,  4.91it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)

  7%|▋         | 13/188 [00:03<00:54,  3.19it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle

In [ ]:
candles_month['AFLT'][0]

65.3

In [ ]:
candles_month['OZON'][0]

4851

In [ ]:
arima_preds_week = get_predicitons_for_arima(candles_week, start_date_week, 60)


  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'

100%|██████████| 3/3 [00:01<00:00,  1.67it/s]

100%|██████████| 29/29 [00:02<00:00, 12.69it/s]

 33%|███▎      | 1/3 [00:00<00:00,  3.37it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)

 67%|██████▋   | 2/3 [00:01<00:00,  2.33it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimi

In [ ]:
arima_preds_day = get_predicitons_for_arima(candles_day, start_date_day, 10)


  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'

 33%|███▎      | 1/3 [00:00<00:01,  1.59it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)

 67%|██████▋   | 2/3 [00:01<00:00,  1.32it/s]/usr/local/lib/python3.7/dist

In [ ]:
from sklearn.utils.validation import check_array

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = check_array(y_true, y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def get_predictions_smooth(train, test, model):
    history = train
    predictions = []
    for i in tqdm(range(len(test))):
        md = model(history)
        model_fit = md.fit()
        output = model_fit.forecast()
        y_hat = output[0]
        predictions.append(y_hat)
        history.append(test[i])
    return predictions

In [ ]:
def get_predicitons_for_smooth(candles, start_date, interval, model):
  predictions = {}
  for key in candles:
    curr_candles = candles[key]
    train_length = int(len(curr_candles) * 0.7)
    train = curr_candles[:train_length]
    test = curr_candles[train_length:]
    predicts = get_predictions_smooth(train, test, model)
    predictions[key] = [test, predicts]
  return predictions

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
smooth_preds_day = get_predicitons_for_smooth(candles_day, start_date_day, 10, SimpleExpSmoothing)
smooth_preds_week = get_predicitons_for_smooth(candles_week, start_date_week, 60, SimpleExpSmoothing)
smooth_preds_month = get_predicitons_for_smooth(candles_month, start_date_month, 60, SimpleExpSmoothing)

In [ ]:
hw_smooth_preds_day = get_predicitons_for_smooth(candles_day, start_date_day, 10, ExponentialSmoothing)
hw_smooth_preds_week = get_predicitons_for_smooth(candles_week, start_date_week, 60, ExponentialSmoothing)
hw_smooth_preds_month = get_predicitons_for_smooth(candles_month, start_date_month, 60, ExponentialSmoothing)

In [ ]:
from arch import arch_model

In [ ]:
am = arch_model(candles_month['AFLT'], vol='Garch', p=1, o=0, q=1, dist='Normal')
am_fit = am.fit()
forecast_am = am_fit.forecast()

In [ ]:
forecast_am.variance.values[-1, :][0]

24.997760104767924

In [ ]:
def get_predictions_garch(train, test):
    history = train
    predictions = []
    for i in tqdm(range(len(test))):
        md = arch_model(history, vol='Garch')
        model_fit = md.fit()
        output = model_fit.forecast()
        y_hat = output.variance.values[-1, :][0]
        predictions.append(y_hat)
        history.append(test[i])
    return predictions

In [ ]:
def get_predicitons_for_garch(candles, start_date, intervall):
  predictions = {}
  for key in candles:
    curr_candles = candles[key]
    train_length = int(len(curr_candles) * 0.7)
    train = curr_candles[:train_length]
    test = curr_candles[train_length:]
    predicts = get_predictions_garch(train, test)
    predictions[key] = [test, predicts]
  return predictions

In [ ]:
garch_preds_day = get_predicitons_for_garch(candles_day, start_date_day, 10)
garch_preds_week = get_predicitons_for_garch(candles_week, start_date_week, 60)
garch_preds_month = get_predicitons_for_garch(candles_month, start_date_month, 60)

In [ ]:
# arima_preds_month
# smooth_preds_month
# hw_smooth_preds_month
# garch_preds_month



In [ ]:
import numpy as np

In [ ]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [ ]:
aflt_mse = []
aflt_mae = []
aflt_mape = []

In [ ]:
# for key in arima_preds_month:
#   print(key)
#   arima_mse_month.append(mean_squared_error(arima_preds_month[key][0],arima_preds_month[key][1]))
#   mae_month.append(mean_absolute_error(arima_preds_month[key][0],arima_preds_month[key][1]))
#   mape_month.append(MAPE(np.array(arima_preds_month[key][0]),np.array(arima_preds_month[key][1])))

In [ ]:
# columns = ['Aeroflot', 'MTS', 'Ozon']
# rows = ['MSE', 'MAE', 'MAPE']
# data = [
#      arima_mse_month, mae_month, mape_month   
# ]

In [ ]:

arrs = [arima_preds_month, smooth_preds_month, hw_smooth_preds_month, garch_preds_month]
for arr in arrs:
  aflt_mse.append(mean_squared_error(arr['AFLT'][0], arr['AFLT'][1]))
  aflt_mae.append(mean_absolute_error(arr['AFLT'][0], arr['AFLT'][1]))
  aflt_mape.append(MAPE(np.array(arr['AFLT'][0]), np.array(arr['AFLT'][1])))

In [ ]:
print('AFLT MSE', str(aflt_mse))
print('AFLT MAE', str(aflt_mae))
print('AFLT MAPE', str(aflt_mape))

AFLT MSE [0.24654433780191673, 0.24179838152646715, 0.24179838152646715, 2554.945647756208]
AFLT MAE [0.280678923271867, 0.25862519369384795, 0.25862519369384795, 49.91343886702036]
AFLT MAPE [0.40061091770890717, 0.36945701707985146, 0.36945701707985146, 71.09302491836527]


In [ ]:
candles_month['OZON'][:]

In [ ]:
def get_predicitons_for_ma(candles, start_date, interval):
  predictions = {}
  for key in candles:
    curr_candles = candles[key]
    # _, best_params, _ = selection_of_arima_params(curr_candles)
    train_length = int(len(curr_candles) * 0.7)
    train = curr_candles[:train_length]
    test = curr_candles[train_length:]
    predicts = get_predictions_arima(train, test, (0,0,1))
    predictions[key] = [test, predicts]
  return predictions

In [ ]:
ma_pred_month = get_predicitons_for_ma(candles_month, start_date_month, 60)


  4%|▎         | 7/188 [00:01<00:26,  6.84it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'

100%|██████████| 188/188 [00:27<00:00,  6.86it/s]

100%|██████████| 188/188 [00:26<00:00,  7.05it/s]

100%|██████████| 188/188 [00:49<00:00,  3.80it/s]


In [ ]:
aflt_mse = []
aflt_mae = []
aflt_mape = []

arrs = [ma_pred_month, arima_preds_month, smooth_preds_month, hw_smooth_preds_month]
for arr in arrs:
  aflt_mse.append(mean_squared_error(arr['AFLT'][0], arr['AFLT'][1]))
  aflt_mae.append(mean_absolute_error(arr['AFLT'][0], arr['AFLT'][1]))
  aflt_mape.append(MAPE(np.array(arr['AFLT'][0]), np.array(arr['AFLT'][1])))

print('AFLT MSE', str(aflt_mse))
print('AFLT MAE', str(aflt_mae))
print('AFLT MAPE', str(aflt_mape))

AFLT MSE [2.911499599692886, 0.24654433780191673, 0.24179838152646715, 0.24179838152646715]
AFLT MAE [1.5878481405950327, 0.280678923271867, 0.25862519369384795, 0.25862519369384795]
AFLT MAPE [2.2469572154521047, 0.40061091770890717, 0.36945701707985146, 0.36945701707985146]


In [ ]:
mts_mse = []
mts_mae = []
mts_mape = []


arrs = [ma_pred_month, arima_preds_month, smooth_preds_month, hw_smooth_preds_month]
for arr in arrs:
  mts_mse.append(mean_squared_error(arr['MTSS'][0], arr['MTSS'][1]))
  mts_mae.append(mean_absolute_error(arr['MTSS'][0], arr['MTSS'][1]))
  mts_mape.append(MAPE(np.array(arr['MTSS'][0]), np.array(arr['MTSS'][1])))

print('MTSS MSE', str(mts_mse))
print('MTSS MAE', str(mts_mae))
print('MTSS MAPE', str(mts_mape))

MTSS MSE [16.084188715978115, 1.359435883459718, 1.3511528822601149, 1.3511528822601149]
MTSS MAE [3.7659831975717277, 0.6451774999145784, 0.6230117803044848, 0.6230117803044848]
MTSS MAPE [1.1274151823899412, 0.19411406598333675, 0.1873905010539883, 0.1873905010539883]


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import median_absolute_error

In [ ]:
def get_predictions_arimax(train, test):
    history = train
    predictions = []
    for i in tqdm(range(len(test))):
        model = SARIMAX(history, order=(1,1, 1), seasonal_order=(0, 0, 0, 0))
        model_fit = model.fit()
        output = model_fit.forecast()
        y_hat = output[0]
        predictions.append(y_hat)
        history.append(test[i])
    return predictions

In [ ]:
def get_predicitons_for_arimax(candles, start_date, interval):
  predictions = {}
  for key in candles:
    curr_candles = candles[key]
    # _, best_params, _ = selection_of_arima_params(curr_candles)
    train_length = int(len(curr_candles) * 0.7)
    train = curr_candles[:train_length]
    test = curr_candles[train_length:]
    predicts = get_predictions_arimax(train, test)
    predictions[key] = [test, predicts]
  return predictions




In [ ]:
pr = get_predicitons_for_arimax(candles_month, start_date_month, 60)


100%|██████████| 188/188 [00:28<00:00,  6.54it/s]

100%|██████████| 188/188 [00:26<00:00,  7.23it/s]

  0%|          | 0/188 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'

100%|██████████| 188/188 [00:29<00:00,  6.34it/s]


In [ ]:
mts_mse = []
mts_mae = []
mts_mape = []


arrs = [pr, arima_preds_month, smooth_preds_month, hw_smooth_preds_month]
for arr in arrs:
  mts_mse.append(mean_squared_error(arr['MTSS'][0], arr['MTSS'][1]))
  mts_mae.append(mean_absolute_error(arr['MTSS'][0], arr['MTSS'][1]))
  mts_mape.append(MAPE(np.array(arr['MTSS'][0]), np.array(arr['MTSS'][1])))

print('MTSS MSE', str(mts_mse))
print('MTSS MAE', str(mts_mae))
print('MTSS MAPE', str(mts_mape))

MTSS MSE [1.3476797116766488, 1.359435883459718, 1.3511528822601149, 1.3511528822601149]
MTSS MAE [0.6231126238216441, 0.6451774999145784, 0.6230117803044848, 0.6230117803044848]
MTSS MAPE [0.18742299152592656, 0.19411406598333675, 0.1873905010539883, 0.1873905010539883]


In [ ]:
smooth_preds_month['AFLT'][0][:5]

[71.32, 71.6, 71.74, 71.34, 70.96]

In [ ]:
hw_smooth_preds_month['AFLT'][0][:5]

[71.32, 71.6, 71.74, 71.34, 70.96]

In [ ]:
aflt_mse = []
aflt_mae = []
aflt_mape = []

arrs = [pr, arima_preds_month, smooth_preds_month, hw_smooth_preds_month]
for arr in arrs:
  aflt_mse.append(mean_squared_error(arr['AFLT'][0], arr['AFLT'][1]))
  aflt_mae.append(mean_absolute_error(arr['AFLT'][0], arr['AFLT'][1]))
  aflt_mape.append(MAPE(np.array(arr['AFLT'][0]), np.array(arr['AFLT'][1])))

print('AFLT MSE', str(aflt_mse))
print('AFLT MAE', str(aflt_mae))
print('AFLT MAPE', str(aflt_mape))

AFLT MSE [0.24147080930203482, 0.24654433780191673, 0.24179838152646715, 0.24179838152646715]
AFLT MAE [0.26050363457349807, 0.280678923271867, 0.25862519369384795, 0.25862519369384795]
AFLT MAPE [0.3721812480527463, 0.40061091770890717, 0.36945701707985146, 0.36945701707985146]


In [ ]:
ozon_mse = []
ozon_mae = []
ozon_mape = []

arrs = [arima_preds_day, smooth_preds_day, hw_smooth_preds_day]
for arr in arrs:
  ozon_mse.append(mean_squared_error(arr['OZON'][0], arr['OZON'][1]))
  ozon_mae.append(mean_absolute_error(arr['OZON'][0], arr['OZON'][1]))
  ozon_mape.append(MAPE(np.array(arr['OZON'][0]), np.array(arr['OZON'][1])))

print('ozon MSE', str(ozon_mse))
print('ozon MAE', str(ozon_mae))
print('ozon MAPE', str(ozon_mape))

ozon MSE [70.0082798606557, 59.51190383311256, 59.51190383311256]
ozon MAE [5.6403889260734665, 5.142857129646624, 5.142857129646624]
ozon MAPE [0.13313272736402193, 0.12139784011151106, 0.12139784011151106]


In [ ]:
arima_preds_month['OZON'][0][-10:]

[4233, 4246.5, 4239.5, 4244, 4206, 4206, 4233, 4246.5, 4239.5, 4244]

In [ ]:
arima_preds_month['OZON'][1][-10:]

[4202.179648228186,
 4232.606041370457,
 4249.79651092902,
 4243.424436396831,
 4243.998454166054,
 4200.935644408396,
 4203.918663949615,
 4234.232913301419,
 4249.467636445982,
 4241.673974005173]

In [ ]:
mts_mse = []
mts_mae = []
mts_mape = []


arrs = [arima_preds_day, smooth_preds_day, hw_smooth_preds_day]
for arr in arrs:
  mts_mse.append(mean_squared_error(arr['MTSS'][0], arr['MTSS'][1]))
  mts_mae.append(mean_absolute_error(arr['MTSS'][0], arr['MTSS'][1]))
  mts_mape.append(MAPE(np.array(arr['MTSS'][0]), np.array(arr['MTSS'][1])))

print('MTSS MSE', str(mts_mse))
print('MTSS MAE', str(mts_mae))
print('MTSS MAPE', str(mts_mape))

MTSS MSE [0.1315843746301302, 0.1259782304265001, 0.1259782304265001]
MTSS MAE [0.2818226181558159, 0.27043668924163117, 0.27043668924163117]
MTSS MAPE [0.08396882736409823, 0.08057991588286131, 0.08057991588286131]


In [ ]:
arr = range(20)

In [ ]:
model = ARIMA([0,1,2,3,4,5,6,7], order=(1, 1, 1))
model_fit = model.fit()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


In [ ]:
model_fit.forecast()

array([7.99991278])

In [ ]:
arr = [4206, 4206, 4221.5, 4237, 4234, 4235, 4233.5, 4233, 4230.5, 4233.5, 4237, 4247, 4233, 4246.5, 4246.5, 4246.5, 4246, 4241.5, 4236.5, 4239.5, 4237, 4240, 4218, 4238, 4239, 4244, 4240.5, 4244, 4248.5, 4247, 4243.5, 4248, 4244, 4243.5, 4235, 4235, 4233, 4239, 4237.5, 4201, 4208, 4180.5, 4144, 4123.5, 4109.5, 4107.5, 4109, 4105, 4101.5, 4102, 4070, 4039, 4056.5, 4040, 4032.5, 4034, 4044, 4039.5, 4024, 4013, 4018.5, 4016.5, 4031.5, 4025.5, 4023, 4022, 4023, 4017, 4011, 4010.5, 4013.5, 4011, 4013, 4010, 4007.5, 4026, 4015, 4012.5, 4015, 4023, 4025, 4027.5, 4031.5, 4035, 4034, 4074, 4089, 4093.5, 4078, 4093, 4087, 4078, 4072.5, 4066.5, 4084.5, 4081, 4079, 4082, 4075.5, 4058.5, 4028.5, 4015.5, 4020, 4031.5, 4045, 4050.5, 4047.5]

In [ ]:
md = ExponentialSmoothing(arr)
md_fit = md.fit()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,


In [ ]:
md_fit.predict()

array([4046.80852294])

In [ ]:
arr.append(4046.80852294)

In [ ]:
md = ExponentialSmoothing(arr, trend='add')
md_fit = md.fit()
md_fit.predict()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,


array([4046.04582802])

In [ ]:
md = ExponentialSmoothing([0,1,2,3,4,5,2,6,4, 4.99999999,4.68513555,4.68513752, )
md_fit = md.fit()
md_fit.predict()

SyntaxError: ignored